In [762]:
import pandas as pd
import numpy as np
import re

In [763]:
steam = pd.read_csv("/Users/jasonchiu0803/Desktop/data_bootcamp/Python Introduction/steam_all/steam_all_new.csv")

In [764]:
steam1 = steam

In [765]:
steam1.shape

(7548, 17)

In [766]:
steam1.columns

Index([u'main_link', u'no_user', u'title', u'url', u'mother_node', u'app_id',
       u'publish', u'user_rating', u'keywords', u'Rdate', u'critics_review',
       u'genre', u'list_title', u'desc', u'type', u'no_links', u'game_spcs'],
      dtype='object')

In [767]:
steam1["no_user"][0]

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t- 79% of the 3,678 user reviews for this game are positive.\t\t\t\t\t\t\t\t\t\t\t'

In [768]:
steam1["no_user"] = steam1["no_user"].str.strip()

In [769]:
#grabbing percentage of positive reviews
steam1["percent"] = steam1["no_user"].str.extract("- (\d{2,3})\% .+")

In [770]:
steam1["percent"] = pd.to_numeric(steam1["percent"])

In [771]:
#grabbing number of reviews
steam1["reivew_users"]=pd.to_numeric(steam1["no_user"].str.extract("- \d{2,3}\% of the (.+) user .+").str.replace(",",""))

In [772]:
#construct a keyword list without extra spaces and lines
for i in range(steam1.shape[0]):
    steam1.loc[i,"keywords_list"] = ', '.join(map(lambda x: x.strip(),str(steam1["keywords"][i]).split(",")))

In [773]:
steam1["type"].value_counts()

None                    4606
Downloadable Content    2920
not_found                 22
Name: type, dtype: int64

In [775]:
#checking if mother game differs from title - these should match with DLC and other type
for i in range(steam1.shape[0]):
    if steam1.loc[i,"title"] != steam1.loc[i,"mother_node"]:
        steam1.loc[i,"sub"] = 1

In [789]:
for i in range(steam1.shape[0]):
    if (steam1.loc[i,"sub"] == 1)&(steam1.loc[i,"type"]!="Downloadable Content"):
        steam1.loc[i,"dlc_notsub"] = 1

In [799]:
steam1["user_rating"].value_counts()

Very Positive              2054
Positive                   1000
Mostly Positive             952
Overwhelmingly Positive     244
Name: user_rating, dtype: int64

In [783]:
#assign not-found to NaN
steam1.replace("not-found",np.NaN, inplace=True)

In [800]:
#breaking genre list
for i in range(steam1.shape[0]):
    steam1.loc[i,"genre_list"] = ", ".join(list(set(str(steam1["genre"][i]).split(","))^set(str(steam1["publish"][i]).split(","))))

In [801]:
steam1.shape

(7548, 24)

In [826]:
steam1["type"].value_counts()

None                    4606
Downloadable Content    2920
not_found                 22
Name: type, dtype: int64

In [828]:
#excluding genre and other out - they will be added later on
a = steam1[(steam1["type"]=="None")].groupby("mother_node").first().reset_index()

In [834]:
a.shape

(4573, 24)

In [836]:
a.rename(columns={"title":"title_steam"},inplace=True)

In [843]:
#creating a database to count the number of DLCs
downloadable = steam1.loc[steam1["type"]=="Downloadable Content"]

In [844]:
downloadable['type'].value_counts()

Downloadable Content    2920
Name: type, dtype: int64

In [845]:
downloadable.columns

Index([u'main_link', u'no_user', u'title', u'url', u'mother_node', u'app_id',
       u'publish', u'user_rating', u'keywords', u'Rdate', u'critics_review',
       u'genre', u'list_title', u'desc', u'type', u'no_links', u'game_spcs',
       u'percent', u'reivew_users', u'keywords_list', u'sub', u'genre_list',
       u'dlc_sub', u'dlc_notsub'],
      dtype='object')

In [847]:
dlc_count = downloadable[["title","mother_node"]].groupby("mother_node").agg(["count"])

In [851]:
dlc_count1 = dlc_count.reset_index()

In [852]:
#making sure that there is no repating values
dlc_count1.columns = dlc_count1.columns.get_level_values(0)

In [860]:
dlc_count1.rename(columns={'title':'no_dlc'}, inplace=True)
dlc_count1.rename(columns={'mother_node':'mother_dlc'}, inplace=True)

In [861]:
dlc_count1.columns

Index([u'mother_dlc', u'no_dlc'], dtype='object')

In [862]:
dlc_count1.shape

(744, 2)

In [864]:
a.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub'],
      dtype='object')

In [866]:
#adding number of dlc in the 
games_dlc=pd.merge(a, dlc_count1, how="left", left_on = "mother_node", right_on = "mother_dlc")

In [867]:
for i in range(games_dlc.shape[0]):
    if pd.isnull(games_dlc.loc[i, "no_dlc"])==True:
        games_dlc.loc[i,"no_dlc"] = 0

In [871]:
games_dlc.loc[games_dlc["no_dlc"]>=1,"dlc_av"] = 1
games_dlc.loc[games_dlc["no_dlc"]==0,"dlc_av"] = 0

In [872]:
games_dlc["dlc_av"].value_counts()

0.0    4035
1.0     538
Name: dlc_av, dtype: int64

In [873]:
games_dlc.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av'],
      dtype='object')

In [874]:
games_dlc.shape

(4573, 27)

In [1148]:
steam_rank = pd.read_csv("/Users/jasonchiu0803/Desktop/data_bootcamp/Python Introduction/ranking_withprice.csv")

In [1149]:
steam_rank_no = steam_rank.groupby(by="title").first().reset_index()

In [1150]:
steam_rank_no.head(3)

,title,rank_no,price
0,,6413,1.99
1,- Red Obsidian Remnant,3567,8.99
2,Double Oik!,3275,1.78


In [1151]:
steam_rank_no.loc[0,"title"]=""

In [1152]:
steam_rank_no1 = steam_rank_no.groupby(by="title").first().reset_index()

In [1153]:
steam_name = steam1[["mother_node","title"]]

In [1154]:
steam_name = steam_name.drop_duplicates()

In [1155]:
steam_name.shape

(7498, 2)

In [1156]:
ranking = pd.merge(steam_name, steam_rank_no1, how="inner",left_on="title",right_on="title")

In [1157]:
ranking.head(2)

,mother_node,title,rank_no,price
0,DRAGON BALL XENOVERSE 2,DRAGON BALL XENOVERSE 2,202,49.99
1,FOR HONOR,FOR HONOR,713,59.99


In [1158]:
ranking.shape

(4790, 4)

In [1160]:
ranking = ranking.sort_values("rank_no")

In [1164]:
price_tag = ranking[["mother_node","price"]].groupby("mother_node").agg("summary")

AttributeError: 'DataFrameGroupBy' object has no attribute 'summary'

In [1163]:
ranking_mother = ranking.sort_values(["rank_no"])[["mother_node","title","rank_no"]].groupby("mother_node").first().reset_index()

In [927]:
ranking_mother.head(3)

,mother_node,title,rank
0,- Red Obsidian Remnant,- Red Obsidian Remnant,2021
1,"""BUTTS: The VR Experience""","""BUTTS: The VR Experience""",8230
2,#Archery,#Archery,2194


In [929]:
ranking_mother.shape

(4663, 3)

In [931]:
games_dlc.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av'],
      dtype='object')

In [930]:
ranking_mother.rename(columns={'mother_node':'mother_rank','title':'title_rank'}, inplace=True)

In [933]:
rank = pd.merge(games_dlc, ranking_mother, how = "inner", left_on = "mother_node", right_on = "mother_rank")

In [936]:
rank.shape

(4456, 30)

In [937]:
sum(pd.isnull(rank["rank"]))

0

In [938]:
sum(pd.isnull(rank["Rdate"]))

5

In [939]:
rank.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av', u'mother_rank', u'title_rank',
       u'rank'],
      dtype='object')

In [940]:
steam_player = pd.read_csv("/Users/jasonchiu0803/Desktop/data_bootcamp/Selenium/users.csv")

In [941]:
steam_player.shape

(7750, 5)

In [954]:
steam_player.columns

Index([u'current', u'all_time', u'app_id', u'24_peak', u'title'], dtype='object')

In [969]:
steam_app = steam1[["mother_node","title","app_id"]]

In [970]:
steam_app = steam_app.drop_duplicates()

In [972]:
steam_app.head(5)

,mother_node,title,app_id
0,DRAGON BALL XENOVERSE 2,DRAGON BALL XENOVERSE 2,454650
1,FOR HONOR,FOR HONOR,304390
2,Dungeon Warfare,Dungeon Warfare,355980
3,theHunter: Call of the Wild,theHunter: Call of the Wild,518790
4,Thimbleweed Park,Thimbleweed Park,569860


In [974]:
name_player = pd.merge(steam_player, steam_app, how="inner",left_on = "app_id",right_on = "app_id")

In [976]:
name_player.columns

Index([u'current', u'all_time', u'app_id', u'24_peak', u'title_x',
       u'mother_node', u'title_y'],
      dtype='object')

In [977]:
name_player.shape

(3453, 7)

In [989]:
name_player_sum = name_player.groupby('mother_node').agg(["sum"]).reset_index()

In [991]:
name_player_sum.head(10)

,mother_node,current,all_time,app_id,24_peak,title_x,title_y
,,sum,sum,sum,sum,sum,sum
0,,2,27,562210,15,Ancient Rus,
1,- Red Obsidian Remnant,10,290,610960,171,Red Obsidian Remnant,- Red Obsidian Remnant
2,#KILLALLZOMBIES,1,11,303720,2,#killallzombies,#KILLALLZOMBIES
3,$1 Ride,8,862,508290,629,$1 Ride,$1 Ride
4,(Tale of Wuxia),36,963,377530,201,(Tale of Wuxia),(Tale of Wuxia)
5,- Arcane Raise -,4,146,603750,6,- Arcane Raise -,- Arcane Raise -
6,/reality,1,10,562280,2,</reality>,/reality
7,1 Moment Of Time: Silentville,4,115,497400,9,1 Moment Of Time: Silentville,1 Moment Of Time: Silentville
8,100% Orange Juice,134,991,282800,212,100% Orange Juice,100% Orange Juice


In [992]:
cols = pd.MultiIndex.from_tuples([("sum", "mother_node"), ("sum", "current"),("sum","all_time"),("sum","app_id"),("sum","24_peak"),("sum","title_x"),("sum","title_y")])

In [993]:
name_player_sum.columns = cols

In [995]:
name_player_sum.columns = name_player_sum.columns.droplevel()

In [997]:
name_player_sum.head(5)

,mother_node,current,all_time,app_id,24_peak,title_x,title_y
0,,2,27,562210,15,Ancient Rus,
1,- Red Obsidian Remnant,10,290,610960,171,Red Obsidian Remnant,- Red Obsidian Remnant
2,#KILLALLZOMBIES,1,11,303720,2,#killallzombies,#KILLALLZOMBIES
3,$1 Ride,8,862,508290,629,$1 Ride,$1 Ride
4,(Tale of Wuxia),36,963,377530,201,(Tale of Wuxia),(Tale of Wuxia)


In [998]:
final_player = pd.merge(rank, name_player_sum, how = "inner",left_on = "mother_node", right_on = "mother_node")

In [999]:
final_player.shape

(3358, 36)

In [1004]:
final_player.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id_x', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av', u'mother_rank', u'title_rank',
       u'rank', u'current', u'all_time', u'app_id_y', u'24_peak', u'title_x',
       u'title_y'],
      dtype='object')

In [1039]:
final_player.head(2)

,mother_node,main_link,no_user,title_steam,url,app_id_x,publish,user_rating,keywords,Rdate,...,dlc_av,mother_rank,title_rank,rank,current,all_time,app_id_y,24_peak,title_x,title_y
0,- Red Obsidian Remnant,http://store.steampowered.com/app/610960/?snr=...,- 93% of the 149 user reviews for this game ar...,- Red Obsidian Remnant,http://store.steampowered.com/app/610960/__Red...,610960,"Red Obsidian Studio,Beijing New Era Network Te...",Very Positive,\r\n\t\t\t\t\t\t\t\t\t\t\t\tEarly Access\t\t\t...,"Apr 20, 2017",...,0.0,- Red Obsidian Remnant,- Red Obsidian Remnant,2021,10,290,610960,171,Red Obsidian Remnant,- Red Obsidian Remnant
1,#KILLALLZOMBIES,http://store.steampowered.com/app/303720/?snr=...,- 40% of the 15 user reviews for this game are...,#KILLALLZOMBIES,http://store.steampowered.com/app/303720/KILLA...,303720,"Beatshapers,8Floor,Beatshapers",NaN,\r\n\t\t\t\t\t\t\t\t\t\t\t\tAction\t\t\t\t\t\t...,"Aug 10, 2016",...,0.0,#KILLALLZOMBIES,#KILLALLZOMBIES,5399,1,11,303720,2,#killallzombies,#KILLALLZOMBIES


In [1040]:
sum(pd.isnull(final_player["Rdate"]))

4

In [1047]:
final_player["str_date"] = final_player["Rdate"].str.contains(",")

In [1079]:
for i in range(len(final_player)):
    if final_player.loc[i, "str_date"] is True:
        final_player.loc[i,"DATE"] = pd.to_datetime(final_player.loc[i,"Rdate"], format = "%b %d, %Y")
    else:
        final_player.loc[i,"DATE"] = pd.to_datetime(final_player.loc[i,"Rdate"], format = "%b %Y")

In [1061]:
final_player.head(2)

,mother_node,main_link,no_user,title_steam,url,app_id_x,publish,user_rating,keywords,Rdate,...,title_rank,rank,current,all_time,app_id_y,24_peak,title_x,title_y,str_date,DATE
0,- Red Obsidian Remnant,http://store.steampowered.com/app/610960/?snr=...,- 93% of the 149 user reviews for this game ar...,- Red Obsidian Remnant,http://store.steampowered.com/app/610960/__Red...,610960,"Red Obsidian Studio,Beijing New Era Network Te...",Very Positive,\r\n\t\t\t\t\t\t\t\t\t\t\t\tEarly Access\t\t\t...,"Apr 20, 2017",...,- Red Obsidian Remnant,2021,10,290,610960,171,Red Obsidian Remnant,- Red Obsidian Remnant,True,2017-04-20
1,#KILLALLZOMBIES,http://store.steampowered.com/app/303720/?snr=...,- 40% of the 15 user reviews for this game are...,#KILLALLZOMBIES,http://store.steampowered.com/app/303720/KILLA...,303720,"Beatshapers,8Floor,Beatshapers",NaN,\r\n\t\t\t\t\t\t\t\t\t\t\t\tAction\t\t\t\t\t\t...,"Aug 10, 2016",...,#KILLALLZOMBIES,5399,1,11,303720,2,#killallzombies,#KILLALLZOMBIES,True,2016-08-10


In [1062]:
final_player.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id_x', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av', u'mother_rank', u'title_rank',
       u'rank', u'current', u'all_time', u'app_id_y', u'24_peak', u'title_x',
       u'title_y', u'str_date', u'DATE'],
      dtype='object')

In [1071]:
import datetime
datetime.date.today()

datetime.date(2017, 5, 4)

In [1084]:
final_player["time_diff"] = datetime.date.today() - final_player["DATE"]

In [1103]:
for i in range(len(final_player)):
    if pd.isnull(final_player["time_diff"]) is False:
        final_player.loc[i, "time_diff_yr"] = final_player.loc[i,"time_diff"].days/365.25

In [1105]:
final_player.columns

Index([u'mother_node', u'main_link', u'no_user', u'title_steam', u'url',
       u'app_id_x', u'publish', u'user_rating', u'keywords', u'Rdate',
       u'critics_review', u'genre', u'list_title', u'desc', u'type',
       u'no_links', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'sub', u'genre_list', u'dlc_sub', u'dlc_notsub',
       u'mother_dlc', u'no_dlc', u'dlc_av', u'mother_rank', u'title_rank',
       u'rank', u'current', u'all_time', u'app_id_y', u'24_peak', u'title_x',
       u'title_y', u'str_date', u'DATE', u'time_diff', u'time_diff_yr'],
      dtype='object')

In [1108]:
final_file = final_player[["mother_node","title_steam","app_id_x","user_rating","critics_review","game_spcs","percent","reivew_users","keywords_list","genre_list","no_dlc","dlc_av","rank","current","all_time","24_peak","DATE","time_diff_yr","time_diff"]]

In [1112]:
final_file.head(2)

,mother_node,title_steam,app_id_x,user_rating,critics_review,game_spcs,percent,reivew_users,keywords_list,genre_list,no_dlc,dlc_av,rank,current,all_time,24_peak,DATE,time_diff_yr,time_diff
0,- Red Obsidian Remnant,- Red Obsidian Remnant,610960,Very Positive,NaN,"Single-player,Steam Achievements,Steam Trading...",93.0,149.0,"Early Access, Rogue-like, Action, Indie, Adven...","Indie, Adventure, RPG, Action, Early Access",0.0,0.0,2021,10,290,171,2017-04-20,0.038330,14 days
1,#KILLALLZOMBIES,#KILLALLZOMBIES,303720,NaN,NaN,"Single-player,Local Co-op,Shared/Split Screen,...",40.0,15.0,"Action, Indie, Zombies","Action, Indie",0.0,0.0,5399,1,11,2,2016-08-10,0.731006,267 days


In [1113]:
final_file.to_csv("/Users/jasonchiu0803/Desktop/data_bootcamp/Python Introduction/cleaned_data.csv")

In [1117]:
keyword_dict = dict()

In [1116]:
final_file.columns

Index([u'mother_node', u'title_steam', u'app_id_x', u'user_rating',
       u'critics_review', u'game_spcs', u'percent', u'reivew_users',
       u'keywords_list', u'genre_list', u'no_dlc', u'dlc_av', u'rank',
       u'current', u'all_time', u'24_peak', u'DATE', u'time_diff_yr',
       u'time_diff'],
      dtype='object')

In [1122]:
for i in range(len(final_file)):
    a = final_file.loc[i,"keywords_list"].split(", ")
    for i in a:
        if i not in keyword_dict:
            keyword_dict[i] = 1
        if i in keyword_dict:
            keyword_dict[i] = keyword_dict[i] + 1

In [1123]:
keyword_dict

{'1980s': 23,
 "1990's": 61,
 '2.5D': 30,
 '2D': 547,
 '2D Fighter': 36,
 '3D Platformer': 51,
 '3D Vision': 39,
 '4 Player Local': 61,
 '4X': 62,
 '6DOF': 9,
 'Abstract': 22,
 'Action': 1758,
 'Action RPG': 146,
 'Action-Adventure': 63,
 'Adventure': 1641,
 'Agriculture': 10,
 'Aliens': 63,
 'Alternate History': 28,
 'America': 15,
 'Animation & Modeling': 24,
 'Anime': 311,
 'Arcade': 182,
 'Arena Shooter': 28,
 'Artificial Intelligence': 3,
 'Assassin': 25,
 'Asynchronous Multiplayer': 10,
 'Atmospheric': 773,
 'Audio Production': 9,
 'Base Building': 78,
 'Based On A Novel': 13,
 'Basketball': 3,
 'Batman': 11,
 "Beat 'em up": 70,
 'Benchmark': 6,
 'Blood': 27,
 'Board Game': 52,
 'Bowling': 3,
 'Building': 162,
 'Bullet Hell': 59,
 'Bullet Time': 18,
 'CRPG': 40,
 'Capitalism': 8,
 'Card Game': 43,
 'Cartoon': 16,
 'Cartoony': 31,
 'Casual': 936,
 'Character Action Game': 14,
 'Character Customization': 85,
 'Chess': 10,
 'Choices Matter': 91,
 'Choose Your Own Adventure': 70,
 'C

In [1140]:
a = pd.DataFrame(keyword_dict.items(),columns = ["keywords","count"]).sort_values("count",ascending= False)

In [1141]:
a.to_csv("./full_keyword.csv",index = False)

In [1134]:
genre_dict = dict()

In [1142]:
for i in range(len(final_file)):
    c = final_file.loc[i,"genre_list"].split(", ")
    for i in a:
        if i not in genre_dict:
            genre_dict[i] = 1
        if i in genre_dict:
            genre_dict[i] = genre_dict[i] + 1

In [1143]:
b = pd.DataFrame(genre_dict.items(),columns = ["genre","count"]).sort_values("count",ascending= False)

In [1144]:
b.to_csv("./full_genre.csv",index = False)

In [1145]:
b

,genre,count
19,count,3359
1,keywords,3359
21,Indie,1861
15,Action,1495
8,Adventure,1204
0,Strategy,866
3,Casual,728
9,RPG,708
13,Simulation,705
17,Early Access,336
